In [1]:
####################################################################
# Scenario1: Temperature Scaling + entropy -> make dynamic ensemble of models which entropy is less than threshold

#step1: find T vector

#step2: find entropy threshold of total

#step3: sum of softmax vector of each good model(under threshold) -> final inference from softmax vector sum

####################################################################
# Scenario2: MC Dropout -> find confident EE -> static ensemble 

#step1: make MC Dropout model

#step2: find confident EE from experiment

#step3: sum of softmax vector of each good model(under threshold) -> final inference from softmax vector sum

####################################################################
# Scenario3: train new block to choose which exit to inference (JUST SCENARIO, NOT VERIFIED)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models,datasets, transforms
from mevit_model import MultiExitViT
from tqdm import tqdm
####################################################################
IMG_SIZE = 224
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dataset_name=dict();dataset_name['cifar10']=datasets.CIFAR10;dataset_name['cifar100']=datasets.CIFAR100;dataset_name['imagenet']=datasets.ImageNet
dataset_outdim=dict();dataset_outdim['cifar10']=10;dataset_outdim['cifar100']=100;dataset_outdim['imagenet']=1000
##############################################################
################ 0. Hyperparameters ##########################
##############################################################
batch_size = 1024
data_choice='cifar100'
mevit_isload=True
mevit_pretrained_path=f'integrated_ee.pth'

backbone_path=f'vit_{data_choice}_backbone.pth'
start_lr=1e-4
max_iter=200

ee_list=[0,1,2,3,4,5,6,7,8,9]#exit list ex) [0,1,2,3,4,5,6,7,8,9]
exit_loss_weights=[1,1,1,1,1,1,1,1,1,1,1]#exit마다 가중치
exit_num=11
##############################################################
transform = transforms.Compose([
        transforms.Resize(IMG_SIZE),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

train_dataset = dataset_name[data_choice](root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = dataset_name[data_choice](root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load the pretrained ViT model from the saved file
pretrained_vit = models.vit_b_16(weights=models.ViT_B_16_Weights.DEFAULT)

if data_choice != 'imagenet':
    pretrained_vit.heads.head = nn.Linear(pretrained_vit.heads.head.in_features, dataset_outdim[data_choice])  # Ensure output matches the number of classes

    # Load model weights
    pretrained_vit.load_state_dict(torch.load(backbone_path))
    pretrained_vit = pretrained_vit.to(device)
#from torchinfo import summary
#summary(pretrained_vit,input_size= (64, 3, IMG_SIZE, IMG_SIZE))

model = MultiExitViT(pretrained_vit,num_classes=dataset_outdim[data_choice],ee_list=ee_list,exit_loss_weights=exit_loss_weights).to(device)
# Assume a pretrained model (replace with your own model)
model.load_state_dict(torch.load('integrated_ee.pth'))  # Load your trained weights

Files already downloaded and verified
Files already downloaded and verified


<All keys matched successfully>

In [2]:
# Initialize lists to store outputs from each exit and labels
output_list_list = [[] for _ in range(exit_num)]
labels_list = []

# Run inference on the test set and collect the logits from each exit
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Collecting logits", leave=False):
        images, labels = images.cuda(), labels.cuda()
        output_list = model(images)  # Get the output from all exits
        
        # Store the output from each exit
        for i in range(exit_num):
            output_list_list[i].append(output_list[i])
        
        # Store the labels
        labels_list.append(labels)

# Concatenate the collected outputs and labels
output_list = [torch.cat(output_list_list[i]) for i in range(exit_num)]
labels = torch.cat(labels_list)

In [3]:
output_list[0].shape

torch.Size([10000, 100])

In [11]:
# 리스트를 바이너리 파일로 저장하기
file_path = 'cache_[tensor([10000,100])x11].pt'
torch.save(output_list, file_path)

In [ ]:
# 저장한 파일을 다시 불러오기
loaded_tensor_list = torch.load(file_path)

# 데이터 확인
print(type(loaded_tensor_list))  # <class 'list'>
print(len(loaded_tensor_list))   # 11
print(loaded_tensor_list[0].shape)  # torch.Size([10000, 100])

<class 'list'>
11
torch.Size([10000, 100])


In [4]:
import torch.nn.functional as F
# Ensemble the outputs from all exits by averaging and applying softmax
# Assuming exit_num is 11
ensemble_logits = sum(output_list) / exit_num
ensemble_probabilities = F.softmax(ensemble_logits, dim=1)

# Now, output_list contains the logits from each exit, labels contains the true labels,
# and ensemble_probabilities contains the ensembled predictions

In [6]:
_, predicted_labels = torch.max(ensemble_probabilities, dim=1)
correct_predictions = (predicted_labels == labels).sum().item()
total_predictions = labels.size(0)
accuracy = correct_predictions / total_predictions * 100

print(f'Final Accuracy: {accuracy:.2f}%')

Final Accuracy: 87.44%
